In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch

In [6]:
import pandas as pd

In [7]:
from openpyxl.utils import coordinate_to_tuple

In [9]:
from train_test_composer import get_train_data

In [11]:
train_df = get_train_data()

train_df["row"] = train_df["coordinate"].apply(lambda x: coordinate_to_tuple(x)[0])
train_df["col"] = train_df["coordinate"].apply(lambda x: coordinate_to_tuple(x)[1])
train_df = train_df.drop(columns=["coordinate"])

to_inverse = [column for column in train_df.columns if str(column).startswith("is_empty")]
for to_inv_col in to_inverse:
    train_df[to_inv_col] = 1 - train_df[to_inv_col]
train_df

,is_empty,is_string,is_merged,is_bold,is_italic,left_border,right_border,top_border,bottom_border,is_filled,horizontal_alignment,left_horizontal_alignment,right_horizontal_alignment,center_horizontal_alignment,wrapped_text,indent,formula,is_header,file_path,sheet_name,row,col
0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet1,1,1
1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet1,1,2
2,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet1,1,3
3,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet1,1,4
4,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11782,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet7,102,21
11783,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet7,102,22
11784,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet7,102,23
11785,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,.\train\Table detection.xlsx,Sheet7,102,24


In [15]:
table_pics = list({k: v.drop(columns=["file_path", "sheet_name"]) 
                   for k, v in train_df.groupby(["file_path", "sheet_name"])}.values())
table_pics

[      is_empty  is_string  is_merged  is_bold  is_italic  ...  indent  formula  is_header  row  col
 0            0          0          0        0          0  ...       0        0          0    1    1
 1            0          0          0        0          0  ...       0        0          0    1    2
 2            0          0          0        0          0  ...       0        0          0    1    3
 3            0          0          0        0          0  ...       0        0          0    1    4
 4            0          0          0        0          0  ...       0        0          0    1    5
 ...        ...        ...        ...      ...        ...  ...     ...      ...        ...  ...  ...
 1555         0          0          0        0          0  ...       0        0          0   65   20
 1556         0          0          0        0          0  ...       0        0          0   65   21
 1557         0          0          0        0          0  ...       0        0          0 

In [28]:
h = 100
w = 100
c = len(train_df.columns) - 4
# tensor: 100x100x114
all_coordinates = pd.DataFrame([(i, j) for i in range(1, h + 1) for j in range(1, w + 1)], columns=["row", "col"])

normalized_table_pics = [all_coordinates.merge(el, on=["row", "col"], how="left").fillna(0) for el in table_pics]
for el in normalized_table_pics:
    el["features"] = el[el.columns.difference(["row", "col", "features", "is_header"])].apply(tuple, axis=1)
    el.drop(columns=el.columns.difference(["row", "col", "features", "is_header"]), inplace=True)

normalized_table_pics

[      row  col  is_header                                           features
 0       1    1        0.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 1       1    2        0.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 2       1    3        0.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 3       1    4        0.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 4       1    5        0.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 ...   ...  ...        ...                                                ...
 9995  100   96        0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 9996  100   97        0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 9997  100   98        0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 9998  100   99        0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 9999  100  100        0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
 
 [10000 rows x 4 columns],
       row  col  is_header         

In [49]:
def df_to_tensor(input_df):
    value_column = input_df.columns.difference(["row", "col"])[0]
    
    def get_df_list(cur_df, column_name):
        return [w.drop(columns=column_name).copy() for v, w in cur_df.sort_values([column_name]).groupby(column_name)]

    fa = get_df_list(test_df, "row")

    sa = [list(map(lambda x: list(x[value_column].iloc[0]), get_df_list(el, "col"))) for el in fa]

    return sa

In [ ]:
x_dataset = [el.drop(columns="is_header") for el in normalized_table_pics]
x_tensors = [df_to_tensor(el) for el in x_dataset]
x_tensors

In [ ]:
y_dataset = [el.drop(columns="features") for el in normalized_table_pics]
y_tensors = [df_to_tensor(el) for el in y_dataset]
y_tensors